In [1]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import pandas as pd

In [14]:
import os
os.path.getsize("./models/dv-2021-08-15.b")

13191

In [6]:
categorical = ['PUlocationID', 'DOlocationID']

In [7]:
df = pd.read_parquet('./data/fhv_tripdata_2021-02.parquet')

def train_model(df, categorical):

    train_dicts = df[categorical].to_dict(orient='records')
    dv = DictVectorizer()
    X_train = dv.fit_transform(train_dicts) 
    y_train = df.duration.values

    print(f"The shape of X_train is {X_train.shape}")
    print(f"The DictVectorizer has {len(dv.feature_names_)} features")

    lr = LinearRegression()
    lr.fit(X_train, y_train)
    y_pred = lr.predict(X_train)
    mse = mean_squared_error(y_train, y_pred, squared=False)
    print(f"The MSE of training is: {mse}")
    return lr, dv

def prepare_features(df, categorical, train=True):
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    mean_duration = df.duration.mean()
    if train:
        print(f"The mean duration of training is {mean_duration}")
    else:
        print(f"The mean duration of validation is {mean_duration}")
    
    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    return df

In [8]:
df = prepare_features(df, categorical, train=True)


The mean duration of training is 16.859265811074096


In [9]:
lr, dv = train_model(df, categorical)

The shape of X_train is (990113, 526)
The DictVectorizer has 526 features
The MSE of training is: 10.958228946973438


In [39]:
# date.strftime("%m")
int(datetime.datetime.strptime(date, "%Y-%m-%d").strftime("%m"))

3

In [58]:
def f(date):
    # eщё надо цикл с годом 
    if date == None:
        m = int(datetime.datetime.now().strftime("%m"))
    else:
        m = int(datetime.datetime.strptime(date, "%Y-%m-%d").strftime("%m"))
        
    if m == 2:
        val_m, train_m = 1, 12
    elif m == 1:
        val_m, train_m = 12, 11
    else: 
        val_m, train_m = m - 1, m - 2
        
    def f(val):

        if val >= 10:
            return str(val)
        else:
            return '0' + str(val)
        
    return f"./data/fhv_tripdata_2021-{f(val_m)}.parquet", f"./data/fhv_tripdata_2021-{f(train_m)}.parquet"

In [63]:
pd.read_parquet(f(date = "2021-08-15")[1])

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00021,2021-06-01 00:40:51,2021-06-01 00:50:22,173.0,173.0,NaN,B00021
1,B00021,2021-06-01 00:51:23,2021-06-01 01:19:01,173.0,36.0,NaN,B00021
2,B00021,2021-06-01 00:07:10,2021-06-01 00:17:34,173.0,129.0,NaN,B00021
3,B00021,2021-06-01 00:22:08,2021-06-01 00:27:22,82.0,82.0,NaN,B00021
4,B00021,2021-06-01 00:59:06,2021-06-01 01:06:58,56.0,173.0,NaN,B00021
...,...,...,...,...,...,...,...
1311341,B03321,2021-06-30 23:00:00,2021-06-30 23:45:00,NaN,NaN,NaN,B03321
1311342,B03321,2021-06-30 23:00:00,2021-06-30 23:35:00,NaN,NaN,NaN,B03323
1311343,B03340,2021-06-30 23:00:00,2021-06-30 23:58:00,NaN,NaN,NaN,B02866
1311344,B03340,2021-06-30 23:10:00,2021-06-30 23:37:00,NaN,NaN,NaN,B03233
